# 패키지 선언

In [1]:
# 패키지 불러오기

import requests
import bs4
from bs4 import BeautifulSoup
import os
import random
import hashlib
import urllib.request
import urllib
from urllib.parse import urlparse, urlencode, parse_qs
import pandas as pd
import csv
import re

from datetime import datetime, timedelta
from urllib import parse
from openpyxl import load_workbook
import time
import requests
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent

from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

ua = UserAgent()
userAgent = ua.random
print(userAgent)

options = Options()
options.add_argument(f"user-agent={userAgent}")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')

executable_path = '/home/ubuntu/book_crawling/geckodriver' # chrome driver의 경로
driver = webdriver.Firefox(executable_path=executable_path, options=options)

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1


# 교보문고 크롤링 함수

In [2]:
def parse_kyobo(isbn):
    
    book = {'isbn' : isbn,
            '제목' : '없음',
            '저자' : '없음',
            '역자' : '없음',
            '별점' : '없음',
            '리뷰' : '없음',
            '카테고리' : '없음',
            '책소개' : '없음',
            '저자소개' : '없음',
            '추천사' : '없음',
            '목차' : '없음',
            '책속으로' : '없음',
            '출판사리뷰' : '없음'
           }
    
    try:
        book_spec = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode='
        url = book_spec + isbn
        driver.get(url)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        spec_tags = soup.select('.box_detail_article')
        
        try:
            buttons = driver.find_elements_by_class_name('btn_small.btn_more')  # 모든 버튼
            more_btns = list(filter(lambda x: '더보기' in x.text, buttons))    # 더보기 버튼
            for btn in more_btns:
                btn.click()
        except:
            pass

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 제목, 저자, 역자
        box_detail = soup.select_one('.box_detail_point')
        title = box_detail.find('strong').text.strip()
        book['제목'] = title
        
        try:
            authors = box_detail.select('.detail_author')
            author = ','.join([auth.text.strip() for auth in authors])
        except:
            author = '없음'
        book['저자'] = author
            
        try:
            translators = box_detail.select('.detail_translator')
            translator = ','.join([tran.text for tran in translators])
        except:
            translator = '없음'
        book['역자'] = translator
        
        # 별점, 리뷰
        try:
            popup = soup.select_one('.popup_load')
            rating = popup.em.text
            review_text = popup.em.nextSibling.strip()
            review_num = re.findall('\d+', review_text)[0]
        except:
            rating = '없음'
            review_num = '없음'
        book['별점'] = rating
        book['리뷰'] = review_num
        
        # 카테고리
        try:
            cate_list = soup.select_one('.list_detail_category').find_all('a')
            cate_texts = [x.text.strip() for x in cate_list]
            category = ' '.join(cate_texts)
        except:
            category = '없음'
        book['카테고리'] = category

        # 책소개
        try:
            ex1 = soup.select_one('.title_detail_basic2').get_text().strip()
            ex2 = soup.select_one('.box_detail_article').get_text().strip()
            try:
                ex3_a = soup.select_one('.box_detail_comment').get_text().strip()
                ex3 = re.sub('[\\n|\\t]{2,}', ' ', ex3_a)
                total_ex = ex1 + '\n\n'+ ex2 + '\n\n' + ex3
            except:
                total_ex = ex1 + '\n\n'+ ex2
        except:
            total_ex = '없음'
        book['책소개'] = total_ex
        
        # 저자 소개
        try:
            box_author = soup.select_one('.box_author')
            authors_el = box_author.find_all('div', attrs={'class':'content', 'style':'display:none;'})
            author_intro = authors_el[0].text.replace('\t', '').replace('닫기', '').strip()  
        except:
            author_intro = '없음'
        book['저자소개'] = author_intro
        
        # 추천사
        try:
            recommend_el = soup.select_one('.box_detail_recommend')
            recommenders = [x.text.strip() for x in recommend_el.find_all('h3')]
            comments_el = recommend_el.find_all('div', {'class' : 'content'})
            comments = [x.text.replace('\t', '').replace('닫기', '').strip().replace('\n\n\n', '\n') for x in comments_el[1::2]]
        except:
            comments = '없음없음'
        book['추천사'] = comments
            
        # 목차, 책속으로, 출판사서평
        table_contents = '없음'
        book_inside = '없음'
        publisher_review = '없음'

        for tag in spec_tags:
            # 목차
            if str(tag.previous_element.previous_element) == '목차':
                try:
                    table_contents = tag.get_text().strip().replace('\n\n', '\n')
                except:
                    table_contents = '없음'
            book['목차'] = table_contents

            # 책속으로
            if str(tag.previous_element.previous_element) == '책 속으로':
                try:
                    inside_book = tag.get_text().strip().replace('\n\n', '\n').replace('\t', '').replace('\n닫기', '')
                    useless_start = inside_book.find('더보기')
                    book_inside = inside_book[useless_start+5:]
                except:
                    book_inside = '없음'
            book['책속으로'] = book_inside

            # 출판사리뷰
            if str(tag.previous_element.previous_element) == '출판사 서평':
                try:
                    publisher_reviews = tag.get_text().replace('\n\n', '\n').replace('\t', '').replace('\n닫기', '')
                    useless_start = publisher_reviews.find('더보기')
                    publisher_review = publisher_reviews[useless_start+5:]
                except:
                    publisher_review = '없음'
            book['출판사리뷰'] = publisher_review
    except:
        return book
        
    return book

In [3]:
isbn = '9788970127248'
parse_kyobo(isbn)

{'isbn': '9788970127248',
 '제목': '총 균 쇠',
 '저자': '재레드 다이아몬드',
 '역자': '김진준',
 '별점': '9.1',
 '리뷰': '555',
 '카테고리': '역사/문화 > 역사일반 > 문명/문명사',
 '책소개': '인류문명의 불균형은 총, 균, 쇠 때문이다!\n\n퓰리처상을 수상한 세계적 석학 재레드 다이아몬드 박사의 『총, 균, 쇠』. ‘왜 어떤 민족들은 다른 민족들의 정복과 지배의 대상으로 전락하고 말았는가?’, ‘왜 각 대륙들마다 문명의 발달 속도에 차이가 생겨났는가?’, ‘인간 사회의 다양한 문명은 어디에서 비롯되었는가?’라는 의문을 명쾌하게 분석하여 그 해답을 제시한다. 특별 증보면 ‘일본인은 어디에서 왔는가’를 추가 수록하여 현대 일본인의 조상이 누구인지를 추적한다.\n\n\n이 책은 모든 인류가 수렵과 채집으로 살아가던 1만 3천 년 전 석기시대로 거슬러 올라간다. 저자는 제국, 지역, 문자, 농작물, 총의 기원뿐만 아니라 각 대륙의 인류 사회가 각기 다른 발전의 길을 걷게 된 원인을 설득력 있게 설명함으로써, 역사에 대한 인종주의적 이론의 허구를 벗겨낸다. 뉴기니와 아메리카 원주민에서부터 현대 유럽인과 일본인에 이르기까지, 세계 각지의 인간 생활에 관한 이야기를 흥미진진하게 풀어낸다.\n\n북소믈리에 한마디! 저자는 민족마다 역사가 다르게 진행된 것은 각 민족의 생물학적 차이 때문이 아닌, 환경적 차이 때문이라고 이야기한다. 다시 말해, 선사시대로부터 환경적으로 유리한 지역에서 살게 된 ‘우연’이 오늘날 문명의 우열을 가리게 되었다는 것이다. 따라서 다이아몬드는 남북아메리카와 아프리카 남단의 원주민과 유라시아의 민족들이 선사시대 때부터 거주 지역이 바뀌었더라면 오늘날의 사정은 정반대가 되었을 것으로 단정하는 등 인류의 역사와 문명을 분석하는 데 있어 흥미롭고 의미 있는 관점을 제시한다.',
 '저자소개': '저자 재레드 다이아몬드(Jared Diamond)는 퓰리처 상을 수상한 세계적 석학. 케임브

# csv파일 불러오기 & 크롤링

In [3]:
book_specs = pd.read_csv('./book_spec_korean_isbns.csv')
isbns = book_specs.loc[:, '0'].tolist()

len(isbns)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2872189

In [ ]:
num = 290000
count_save = num - 10000
results = []
for isbn in tqdm(isbns[num-10000:num]):
    time.sleep(1)
    try:
        result = parse_kyobo(isbn)
        results.append(result)
    except:
        result = {'isbn' : isbn,
                  '제목' : '없음',
                  '저자' : '없음',
                  '역자' : '없음',
                  '별점' : '없음',
                  '리뷰' : '없음',
                  '카테고리' : '없음',
                  '책소개' : '없음',
                  '저자소개' : '없음',
                  '추천사' : '없음',
                  '목차' : '없음',
                  '책속으로' : '없음',
                  '출판사리뷰' : '없음'
                 }
        results.append(result)

    count_save += 1
    
    if count_save % 1000 == 0:
        file_name = f'kyobo_{count_save}_287,000.csv'
        results_df = pd.DataFrame(results)
        results_df.to_csv('./crawled_csv/' + file_name, encoding='utf-8-sig')
        results = []
        print(f'saving {count_save}th results to {file_name}')

saving 281000th results to kyobo_281000_287,000.csv
saving 282000th results to kyobo_282000_287,000.csv
saving 283000th results to kyobo_283000_287,000.csv
